In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv("/content/drive/My Drive/DataThesis/BETN073/working_data.csv")

In [ ]:
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Select columns to normalize
columns_to_normalize = ['Concentration']

# Fit and transform the selected columns
data_normalized = data.copy()  # Make a copy to avoid modifying the original data
data_normalized[columns_to_normalize] = scaler.fit_transform(data_normalized[columns_to_normalize])

data_normalized = data_normalized[data_normalized['Year'] != 2008]
data_normalized = data_normalized.reset_index()
data_normalized = data_normalized.drop(columns='index')

# Display the normalized data
print(data_normalized)

      Year  Month  Day  Concentration  DayOfWeek  Weekend
0     2009      1    1       0.041503          4        0
1     2009      1    2       0.019506          5        0
2     2009      1    3       0.027512          6        1
3     2009      1    4       0.069484          0        1
4     2009      1    5       0.137438          1        0
...    ...    ...  ...            ...        ...      ...
3983  2019     12   27       0.174413          5        0
3984  2019     12   28       0.146468          6        1
3985  2019     12   29       0.134440          0        1
3986  2019     12   30       0.254002          1        0
3987  2019     12   31       0.086472          2        0

[3988 rows x 6 columns]


In [ ]:
input_features = data_normalized.drop(columns=['Concentration'])  # Drop the output column
output_measurements = data_normalized['Concentration']

In [ ]:
# Convert DataFrames to numpy arrays
input_features = input_features.to_numpy()
output_measurements = output_measurements.to_numpy()

In [ ]:
output_measurements

array([0.04150266, 0.01950557, 0.02751208, ..., 0.13444007, 0.25400241,
       0.08647238])

In [ ]:
input_features

array([[2009,    1,    1,    4,    0],
       [2009,    1,    2,    5,    0],
       [2009,    1,    3,    6,    1],
       ...,
       [2019,   12,   29,    0,    1],
       [2019,   12,   30,    1,    0],
       [2019,   12,   31,    2,    0]])

In [ ]:
def build_generator(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dense(32, activation='relu'),
        Dense(1)  # Output layer for the generated output measurement
    ])
    return model

def build_discriminator():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(1,)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')  # Output layer for binary classification (real or fake)
    ])
    return model

# Define GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [ ]:
def train_gan(input_features, output_measurements, epochs=1000, batch_size=32):
    for epoch in range(epochs):
        # Get the shape of input features
        input_shape = (input_features.shape[1],)  # Assuming input_features is a 2D array

        # Build the generator and discriminator using the input__shape
        generator = build_generator(input_shape)
        discriminator = build_discriminator()
        gan = build_gan(generator, discriminator)
        # Train discriminator
        generator.compile(loss='mse', optimizer='adam')
        discriminator.compile(loss='binary_crossentropy', optimizer='adam')
        gan.compile(loss='binary_crossentropy', optimizer='adam')

        for epoch in range(epochs):
        # Train discriminator
          fake_output = generator.predict(input_features)
          discriminator_loss_fake = discriminator.train_on_batch(fake_output, np.zeros((len(output_measurements), 1)))
          discriminator_loss_real = discriminator.train_on_batch(output_measurements, np.ones((len(output_measurements), 1)))
          discriminator_loss = 0.5 * np.add(discriminator_loss_fake, discriminator_loss_real)

          # Train generator
          gan_loss = gan.train_on_batch(input_features, np.ones((len(output_measurements), 1)))

          # Print progress
          if epoch % 100 == 0:
            print(f"Epoch: {epoch}, Discriminator Loss: {discriminator_loss}, Generator Loss: {gan_loss}")

    return generator

In [ ]:
# Assuming input_features and output_measurements are numpy arrays
input_features_shape = input_features.shape
print("Shape of input_features:", input_features_shape)

Shape of input_features: (3988, 5)


In [ ]:
generator = train_gan(input_features, output_measurements)

125/125 [==============================] - 0s 1ms/step


InvalidArgumentError: Graph execution error:

Detected at node sequential_1/dense_3/MatMul defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/p